# Marching Wagon of useful Modules


#### Changing you life one package at a time....

### Retrying your functions

In [4]:
# retrying
# https://pypi.python.org/pypi/retrying

import time
import random
from retrying import retry

@retry(stop_max_delay=1000)
def do_something_unreliable():
    if random.randint(0, 10) > 2:
        print "broken"
        raise IOError("Broken sauce, everything is hosed!!!")
    else:
        return "Awesome sauce!"

print do_something_unreliable()


broken
broken
broken
broken
broken
Awesome sauce!


### Waiting for socket and files/direcoties

In [5]:
# python-wait
# https://github.com/shawnsi/python-wait

import wait

# wait for a file to be created
if wait.log.exists('c:/Users/ifruchte/.ssh/id_rsa.pub', timeout=0.5):
    print "directory/file exist"

# wait until I can open a port to google
if wait.tcp.open(80, host='www.google.com', timeout=5):
    print "port is ready"

directory/file exist
port is ready


### dpath - Look/search into nested dicts

In [6]:
# dpath
# https://pypi.python.org/pypi/dpath

import pprint
import dpath.util

x = { "a": { "b": { "3": 2,
                    "43": 30,
                    "c": [],
                    "d": ['red', 'buggy', 'bumpers'],} } }

print "a/b/43 = %s" % dpath.util.get(x, '/a/b/43')

result = dpath.util.search(x, "a/b/[cd]")
pprint.pprint(result)


a/b/43 = 30
{'a': {'b': {'c': [], 'd': ['red', 'buggy', 'bumpers']}}}


### deepdiff - compare dicts easily 

In [7]:
# deepdiff
# http://deepdiff.readthedocs.io/en/latest/
from pprint import pprint
from deepdiff import DeepDiff

t1 = {1:1, 3:3, 4:4, 7:{"a":"hello", 
                        "b":"world", 
                        "c": "world!\nGoodbye!\n1\n2\nEnd"}}
t2 = {1:1, 3:3, 5:5, 6:6, 7:{"a":"hello", 
                             "b":"world!!!", 
                             "c": "world!\n2\nEnd"}}

ddiff = DeepDiff(t1, t2, verbose_level=2)
pprint(ddiff, indent=2)
# if it's multi line, we have unified diff
print ddiff['values_changed']["root[7]['c']"]['diff']

{ 'dictionary_item_added': { 'root[5]': 5, 'root[6]': 6},
  'dictionary_item_removed': { 'root[4]': 4},
  'values_changed': { "root[7]['b']": { 'new_value': 'world!!!',
                                        'old_value': 'world'},
                      "root[7]['c']": { 'diff': '--- \n+++ \n@@ -1,5 +1,3 @@\n world!\n-Goodbye!\n-1\n 2\n End',
                                        'new_value': 'world!\n2\nEnd',
                                        'old_value': 'world!\nGoodbye!\n1\n2\nEnd'}}}
--- 
+++ 
@@ -1,5 +1,3 @@
 world!
-Goodbye!
-1
 2
 End


### tqdm - progressbars for the lazy

In [8]:
# tqdm
# https://pypi.python.org/pypi/tqdm
from time import sleep
from tqdm import tqdm, trange

for i in trange(15):
    sleep(0.1)
    
from tqdm import tnrange, tqdm_notebook

for i in tnrange(5, desc='1st loop'):
    for j in tqdm_notebook(xrange(100), desc='2nd loop'):
        sleep(0.01)


100%|██████████| 15/15 [00:01<00:00,  9.61it/s]


#### Cilck - Building command line applications

In [9]:
# click
# http://click.pocoo.org/

import click

@click.command()
@click.option('--count', default=1, help='Number of greetings.')
@click.option('--name', prompt='Your name',
              help='The person to greet.')
def hello(count, name):
    """Simple program that greets NAME for a total of COUNT times."""
    for x in range(count):
        click.echo('Hello %s!' % name)

        
if __name__ == '__main__':
    # some boiler plate for running in IPython/Jupyter
    # http://click.pocoo.org/5/testing/#basic-testing
    from click.testing import CliRunner
        
    result = CliRunner().invoke(hello, ["--help"])
    print result.output
    
    result = CliRunner().invoke(hello, ["--name", "Israel", "--count", "4"])
    print result.output

    # this is how you usally do it
    # hello()

Usage: hello [OPTIONS]

  Simple program that greets NAME for a total of COUNT times.

Options:
  --count INTEGER  Number of greetings.
  --name TEXT      The person to greet.
  --help           Show this message and exit.

Hello Israel!
Hello Israel!
Hello Israel!
Hello Israel!



### construct - parse/build binary data

In [11]:
# construct
# http://construct.readthedocs.io/en/latest/

from construct import *
from binascii import unhexlify, hexlify
import six


tcp_header = Struct("tcp_header",
    UBInt16("source"),
    UBInt16("destination"),
    UBInt32("seq"),
    UBInt32("ack"),
    EmbeddedBitStruct(
        ExprAdapter(Nibble("header_length"), 
            encoder = lambda obj, ctx: obj / 4,
            decoder = lambda obj, ctx: obj * 4,
        ),
        Padding(3),
        Struct("flags",
            Flag("ns"),
            Flag("cwr"),
            Flag("ece"),
            Flag("urg"),
            Flag("ack"),
            Flag("psh"),
            Flag("rst"),
            Flag("syn"),
            Flag("fin"),
        ),
    ),
    UBInt16("window"),
    UBInt16("checksum"),
    UBInt16("urgent"),
    Field("options", lambda ctx: ctx.header_length - 20),
)

if __name__ == "__main__":
    cap = unhexlify(six.b("0db5005062303fb21836e9e650184470c9bc0000"))
    
    obj = tcp_header.parse(cap)
    print (obj)
    
    obj.destination = 22
    
    built = tcp_header.build(obj)
    print (hexlify(built))
    
    assert cap == built



Container:
    source = 3509
    destination = 80
    seq = 1647329202
    ack = 406252006
    header_length = 20
    flags = Container:
        ns = False
        cwr = False
        ece = False
        urg = False
        ack = True
        psh = True
        rst = False
        syn = False
        fin = False
    window = 17520
    checksum = 51644
    urgent = 0
    options = ''
0db5001662303fb21836e9e650184470c9bc0000


AssertionError: 

### rpyc - seemless remote procudre 

In [ ]:
# rypc
# https://rpyc.readthedocs.io/en/latest/

# server side (let say it's ubuntu)
# $ python rypc_classic.py
# [SLAVE     INFO     14:22:27 tid=2332] server started on 0.0.0.0:18812

# client code
import rpyc
conn = rpyc.classic.connect("rdkucs1.il.nds.com") # our server

remote_sys = conn.modules.sys # access the sys module on the server

remote_os = conn.modules["os"]

remote_os.getcwd() # output: "/home/users/ifruchte/"

